In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "DNT": "1"
    "Upgrade-Insecure-Requests": "1"
}

web = requests.get("https://www.amazon.in/s?k=mobiles&crid=3EQ81A827JPNT&sprefix=mobile%2Caps%2C1004&ref=nb_sb_noss_1", headers=headers)
print(web.status_code)


200


In [ ]:
soup = BeautifulSoup(web.content, "html.parser")

In [ ]:
soup.title

<title>Amazon.in : mobiles</title>

In [ ]:
data = soup.find("div" , class_ = "a-section a-spacing-small a-spacing-top-small")

In [ ]:
len(data)

5

In [ ]:
len(data.find_all("a" , class_ = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"))

0

In [ ]:
items = soup.find_all('span', class_='a-size-medium a-color-base a-text-normal')

In [ ]:
product_elements = soup.find_all('span', {'class': 'a-size-medium a-color-base a-text-normal'})

#Making List to Store data

In [ ]:
mobile_names = []
mobile_features = []

#Use for loop to iterate thow all product

In [ ]:
for product_element in product_elements:
    # Extract the product title text
    product_title = product_element.get_text(strip=True)

    # Check if the title has features in parentheses
    if "(" in product_title:
        name, features_str = product_title.split("(", 1)
        mobile_names.append(name.strip())

        # Initialize a dictionary to store features
        feature_dict = {}

        # Clean and split features
        features = features_str.replace(")", "").split("|")

        # Step 5: Further split features into categories like color, RAM, etc.
        for feature in features:
            feature = feature.strip()

            # Check for common features like color, RAM, etc.
            if "GB RAM" in feature:
                feature_dict['RAM'] = feature.split("GB RAM")[0].strip() + "GB"
            elif "GB Storage" in feature:
                feature_dict['Storage'] = feature.split("GB Storage")[0].strip() + "GB"
            elif "Color" in feature or "Green" in feature or "Blue" in feature or "Black" in feature:  # Expandable for more colors
                feature_dict['Color'] = feature
            elif "Battery" in feature:
                feature_dict['Battery'] = feature
            else:
                # If it doesn't match known features, store it in 'Other'
                feature_dict.setdefault('Other', []).append(feature)

        # Append the features dictionary to mobile_features
        mobile_features.append(feature_dict)

# Step 6: Create a pandas DataFrame from the extracted data
df = pd.DataFrame({
    "Mobile Name": mobile_names,
    "Features": mobile_features
})

In [ ]:
print(df)

                  Mobile Name  \
0          Samsung Galaxy M05   
1             realme GT 6T 5G   
2         realme NARZO N65 5G   
3         realme NARZO 70x 5G   
4                   Redmi 13C   
5            realme NARZO N61   
6    OnePlus Nord CE4 Lite 5G   
7         realme NARZO 70x 5G   
8                  POCO M6 5G   
9   OnePlus Nord CE 3 Lite 5G   
10           realme NARZO N63   
11        realme NARZO N65 5G   
12        realme NARZO N65 5G   
13      Samsung Galaxy M35 5G   
14        realme NARZO 70x 5G   
15        realme NARZO 70x 5G   

                                             Features  
0   {'RAM': 'Mint Green, 4GB', 'Other': ['50MP Dua...  
1   {'RAM': 'Fluid Silver,8GB', 'Other': ['India's...  
2   {'RAM': 'Amber Gold 6GB', 'Other': ['Ultra Sli...  
3   {'RAM': 'Ice Blue, 6GB', 'Other': ['120Hz Ultr...  
4   {'RAM': 'Starfrost White, 6GB', 'Other': ['Pow...  
5   {'RAM': 'Voyage Blue,6GB', 'Other': ['IP54 Dus...  
6                        {'RAM': 'Super Silver

In [ ]:
df.to_csv('mobiles_with_detailed_features.csv', index=True)